Marcin Wardyński  
czwartek, 8:00

### Laboratorium 5: Klasyfikacja

In [5]:
import lab5_utils as utils

corpus, queries, qrels = utils.load_fiqa()

In [2]:
#TODO check the intersection between test, validation and test. Reconsider sizes of the datasets.

In [6]:
from transformers import AutoTokenizer

pl_tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
separator = pl_tokenizer.sep_token

In [7]:

vate_query_to_corpus_dict = utils.prepare_fiqa_qrels(qrels, ['validation', 'test'])
no_vate_corpus_dict = utils.prepare_fiqa_corpus_related_to_selected_subsets(corpus, vate_query_to_corpus_dict, False)

In [8]:
train_index_name = "mw_nlp_lab5_train"
fts_url, train_index_url = utils.create_fts_index(train_index_name)
utils.bulk_load(fts_url, train_index_name, no_vate_corpus_dict)

Failed to delete index 'mw_nlp_lab5_train': {"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [mw_nlp_lab5_train]","resource.type":"index_or_alias","resource.id":"mw_nlp_lab5_train","index_uuid":"_na_","index":"mw_nlp_lab5_train"}],"type":"index_not_found_exception","reason":"no such index [mw_nlp_lab5_train]","resource.type":"index_or_alias","resource.id":"mw_nlp_lab5_train","index_uuid":"_na_","index":"mw_nlp_lab5_train"},"status":404}
Index created.
All documents indexed successfully.


In [9]:
va_query_to_corpus_dict = utils.prepare_fiqa_qrels(qrels, ['validation'])
va_corpus_dict = utils.prepare_fiqa_corpus_related_to_selected_subsets(corpus, va_query_to_corpus_dict, True)

validation_index_name = "mw_nlp_lab5_validation"
fts_url, validation_index_url = utils.create_fts_index(validation_index_name)
utils.bulk_load(fts_url, validation_index_name, va_corpus_dict)

Failed to delete index 'mw_nlp_lab5_validation': {"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [mw_nlp_lab5_validation]","resource.type":"index_or_alias","resource.id":"mw_nlp_lab5_validation","index_uuid":"_na_","index":"mw_nlp_lab5_validation"}],"type":"index_not_found_exception","reason":"no such index [mw_nlp_lab5_validation]","resource.type":"index_or_alias","resource.id":"mw_nlp_lab5_validation","index_uuid":"_na_","index":"mw_nlp_lab5_validation"},"status":404}
Index created.
All documents indexed successfully.


In [10]:
from datasets import Dataset

def collect_training_dataset(index_url, corpus_dict, subsets, separator):
    query_to_corpus_dict = utils.prepare_fiqa_qrels(qrels, subsets)
    queries_dict = utils.prepare_fiqa_queries_for_selected_subset(queries, query_to_corpus_dict)

    results = []

    for q_id, q_text in queries_dict.items():
        collect_passages_for_query(q_id, q_text, corpus_dict, query_to_corpus_dict, index_url, results, separator)

    return Dataset.from_list(results)

def collect_passages_for_query(q_id, q_text, p_dict, q_to_p_dict, index_url, results, separator):
    q_p_ids = list(q_to_p_dict[q_id].keys())

    for q_p_id in q_p_ids:
        results.append(convert_to_json(q_text, p_dict[q_p_id], 1, separator))
        
    fts_results = utils.find_for_phrase_with_exclusion(index_url, q_text, 'text', 3*len(q_p_ids), q_p_ids)
    for fts_result_id in fts_results:
        results.append(convert_to_json(q_text, p_dict[fts_result_id], 0, separator))

def convert_to_json(q_text, p_text, label, separator):
    return {
            "text": f"{q_text}{separator}{p_text}",
            "label": label,
        }
        

In [11]:
train_dataset = collect_training_dataset(train_index_url, no_vate_corpus_dict, ['train'], separator)

In [12]:
validation_dataset = collect_training_dataset(validation_index_url, va_corpus_dict, ['validation'], separator)

In [13]:
from datasets import DatasetDict

datasets = DatasetDict({"train": train_dataset, "validation": validation_dataset})
datasets.save_to_disk("./question-passage-classification")


Saving the dataset (1/1 shards): 100%|██████████| 4952/4952 [00:00<00:00, 483961.91 examples/s]


### Przygotowanie modelu

In [14]:
def tokenize_function(examples):
    return pl_tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = datasets.map(tokenize_function, batched=True)
tokenized_datasets["train"]

Map: 100%|██████████| 4952/4952 [00:01<00:00, 4339.01 examples/s]


Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 56664
})

In [15]:
example = tokenized_datasets["train"][0]
print(example["text"])
print(example["input_ids"])
print(example["attention_mask"])
print("|".join(pl_tokenizer.convert_ids_to_tokens(list(example["input_ids"]))))

print(len([e for e in example["input_ids"] if e != 1]))
print(len([e for e in example["attention_mask"] if e == 1]))

Co jest uważane za wydatek służbowy w podróży służbowej?</s>Wytyczne IRS dotyczące tematu. Ogólnie rzecz biorąc, najlepsze, co mogę powiedzieć, to to, że Twój wydatek biznesowy może podlegać odliczeniu. Ale to zależy od okoliczności i tego, co chcesz odliczyć. Podróże Podatnicy, którzy wyjeżdżają z domu w celach służbowych, mogą odliczyć związane z tym wydatki, w tym koszty dotarcia do miejsca docelowego, koszty zakwaterowania i wyżywienia oraz inne zwykłe i niezbędne wydatki. Podatnicy są uważani za „wyjeżdżających poza dom”, jeśli ich obowiązki wymagają od nich przebywania poza domem znacznie dłużej niż zwykły dzień pracy i muszą spać lub odpoczywać, aby sprostać wymogom pracy. Można odliczyć rzeczywisty koszt posiłków i nieprzewidziane wydatki lub skorzystać ze standardowej diety żywieniowej i obniżonych wymogów ewidencji. Niezależnie od zastosowanej metody odliczenia posiłków są zazwyczaj ograniczone do 50 procent, jak wspomniano wcześniej. Jako koszt można zgłaszać tylko rzeczywis

In [29]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "allegro/herbert-base-cased", num_labels=2
)

model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [30]:
from transformers import TrainingArguments

arguments = TrainingArguments(
    output_dir="./output",
    do_train=True,
    do_eval=True,
    eval_strategy="steps",
    eval_steps=300,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-05,
    num_train_epochs=1,
    logging_first_step=True,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="steps",
    save_steps=300,
    fp16=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=3,
)

#load_best_model_at_end=True,     # Load the best model at the end of training
#metric_for_best_model="accuracy" # Metric to use for the best model
#save_total_limit
#check different bach_size: 32, or 64.

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [31]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [32]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42),
    eval_dataset=tokenized_datasets["validation"].shuffle(seed=42),
    compute_metrics=compute_metrics,
)

In [33]:
%load_ext tensorboard
%tensorboard --logdir ./output/runs
%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 11113), started 0:32:19 ago. (Use '!kill 11113' to kill it.)

In [34]:
trainer.train()

  8%|▊         | 300/3542 [32:07<5:47:13,  6.43s/it]
                                                  

  0%|          | 1/3542 [00:05<5:35:35,  5.69s/it]  


{'loss': 0.7393, 'grad_norm': 9.898111343383789, 'learning_rate': 4.9985883681535854e-05, 'epoch': 0.0}


                                                   

  1%|▏         | 50/3542 [01:49<2:01:15,  2.08s/it] 


{'loss': 0.5806, 'grad_norm': 2.1417925357818604, 'learning_rate': 4.9294184076792775e-05, 'epoch': 0.01}


                                                    

  3%|▎         | 100/3542 [03:33<1:59:03,  2.08s/it]


...

                                                     

 99%|█████████▉| 3500/3542 [2:44:30<01:26,  2.07s/it] 


{'loss': 0.3588, 'grad_norm': 5.945509433746338, 'learning_rate': 5.928853754940711e-07, 'epoch': 0.99}


                                                     

100%|██████████| 3542/3542 [2:46:03<00:00,  2.81s/it]

{'train_runtime': 9963.7699, 'train_samples_per_second': 5.687, 'train_steps_per_second': 0.355, 'train_loss': 0.42961667110393575, 'epoch': 1.0}


TrainOutput(global_step=3542, training_loss=0.42961667110393575, metrics={'train_runtime': 9963.7699, 'train_samples_per_second': 5.687, 'train_steps_per_second': 0.355, 'total_flos': 1.490892484091904e+16, 'train_loss': 0.42961667110393575, 'epoch': 1.0})

## Wyprobuj
- f1 do uczenia
- nie separator </s> tylko tak, jak w przykładzie: pytanie: kontekst:
- nie zawsze 3x zła 1x dobra odp

In [35]:
model.save_pretrained('./best_model_f1')
pl_tokenizer.save_pretrained('./best_model_f1')

('./best_model_f1/tokenizer_config.json',
 './best_model_f1/special_tokens_map.json',
 './best_model_f1/vocab.json',
 './best_model_f1/merges.txt',
 './best_model_f1/added_tokens.json',
 './best_model_f1/tokenizer.json')

In [36]:
test_query_to_corpus_dict = utils.prepare_fiqa_qrels(qrels, ['test'])
test_corpus_dict = utils.prepare_fiqa_corpus_related_to_selected_subsets(corpus, test_query_to_corpus_dict, True)

test_index_name = "mw_nlp_lab5_test"
fts_url, test_index_url = utils.create_fts_index(test_index_name)
utils.bulk_load(fts_url, test_index_name, test_corpus_dict)

Failed to delete index 'mw_nlp_lab5_test': {"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [mw_nlp_lab5_test]","resource.type":"index_or_alias","resource.id":"mw_nlp_lab5_test","index_uuid":"_na_","index":"mw_nlp_lab5_test"}],"type":"index_not_found_exception","reason":"no such index [mw_nlp_lab5_test]","resource.type":"index_or_alias","resource.id":"mw_nlp_lab5_test","index_uuid":"_na_","index":"mw_nlp_lab5_test"},"status":404}
Index created.
All documents indexed successfully.


In [37]:
test_dataset = collect_training_dataset(test_index_url, test_corpus_dict, ['test'], separator)

In [38]:
tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)

results = trainer.evaluate(tokenized_test_datasets)
print(results)

100%|██████████| 427/427 [04:55<00:00,  1.45it/s]

{'eval_loss': 0.5758367776870728, 'eval_accuracy': 0.738569753810082, 'eval_precision': 0.4785005512679162, 'eval_recall': 0.5087924970691676, 'eval_f1': 0.49318181818181817, 'eval_runtime': 300.216, 'eval_samples_per_second': 22.73, 'eval_steps_per_second': 1.422, 'epoch': 1.0}


In [68]:
import torch

input = pl_tokenizer("Sprzedajesz w serwisie eBay bez PayPal?", padding="max_length", truncation=True)

with torch.no_grad():
    outputs = model(**input)


AttributeError: 'list' object has no attribute 'size'

In [35]:
all_corpus_dict = utils.prepare_fiqa_corpus_related_to_selected_subsets(corpus, {}, False)

all_index_name = "mw_nlp_lab5_all"
fts_url, all_index_url = utils.create_fts_index(all_index_name)
utils.bulk_load(fts_url, all_index_name, all_corpus_dict)

Failed to delete index 'mw_nlp_lab5_all': {"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [mw_nlp_lab5_all]","resource.type":"index_or_alias","resource.id":"mw_nlp_lab5_all","index_uuid":"_na_","index":"mw_nlp_lab5_all"}],"type":"index_not_found_exception","reason":"no such index [mw_nlp_lab5_all]","resource.type":"index_or_alias","resource.id":"mw_nlp_lab5_all","index_uuid":"_na_","index":"mw_nlp_lab5_all"},"status":404}
Index created.
All documents indexed successfully.


In [ ]:
test_query_to_corpus_dict = utils.prepare_fiqa_qrels(qrels, ['test'])
test_queries_dict = utils.prepare_fiqa_queries_for_selected_subset(queries, test_query_to_corpus_dict)

def rerank(texts, model):
    inputs = pl_tokenizer(
        texts,
        padding="max_length",
        truncation=True,  # Truncate to the model's max length
        return_tensors="pt"  # Return PyTorch tensors

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    
)

utils.calculate_ndcgs(test_queries_dict, test_query_to_corpus_dict, all_index_url, 'text', 10)

[0.0,
 0.20438239758848611,
 0.2640681225725909,
 0.0,
 0.5,
 0.0,
 0.0,
 0.6131471927654584,
 0.16716045496620227,
 1.0,
 0.3333333333333333,
 0.0,
 0.0,
 0.0,
 0.18495352221149364,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.09478836436955078,
 0.09010000865236768,
 0.7977228895450266,
 0.6309297535714575,
 0.0,
 0.0,
 0.6131471927654584,
 0.0,
 0.08671382849411972,
 0.6309297535714575,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.21109268758922906,
 0.0,
 0.0,
 0.5802792108518124,
 0.21840743681816419,
 0.6992148198508501,
 0.0,
 0.0,
 0.3065735963827292,
 0.2960819109658652,
 0.20438239758848611,
 0.11751610475642714,
 0.0,
 0.16812753627111746,
 0.0,
 0.0,
 0.3333333333333333,
 0.5,
 0.6309297535714575,
 0.5689918564790247,
 0.0,
 0.0,
 0.2890648263178879,
 0.0,
 0.0,
 0.0,
 0.43830722522726884,
 0.19342640361727081,
 0.0,
 0.0,
 0.24630238874073,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.46927872602275644,
 0.7653606369886217,
 0.0,
 1.0,
 0.0,
 0.5969412532155237,
 0.0,
 0.0,
 1.0,
 0.1934264

In [62]:
import importlib
import lab5_utils
importlib.reload(lab5_utils)

<module 'lab5_utils' from '/Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab5/lab5_utils.py'>